![imagen](./img/python.jpg)

# SQL en Python
Podrás atacar a una base de datos SQL desde muchas plataformas/lenguajes. Por supuesto, Python es uno de ellos. Para ello **utilizaremos el módulo `sqlite3`**, que viene incluido en la suite de paquetes de Anaconda.


1. [Configuración del entorno](#1.-Configuración-del-entorno)
2. [Modelo de datos](#2.-Modelo-de-datos)
3. [Queries](#3.-Queries)

    3.1 [Primera query](#3.1-Primera-query)
    
    3.2 [SELECT](#3.2-SELECT)
    
    3.3 [LIMIT](#3.3-LIMIT)
    
    3.4 [DISTINCT](#3.4-DISTINCT)
    
    3.5 [WHERE](#3.5-WHERE)
    
    3.6 [ORDER BY](#3.6-ORDER-BY)
    
    3.7 [Agregaciones](#3.7-Agregaciones)
    
    3.8 [GROUP BY](#3.8-GROUP-BY)
    
    3.9 [JOIN](#3.9-JOIN)
    
    3.10 [VIEW y DROP](#3.10-VIEW-y-DROP)
    
4. [Errores](#4.-Errores)

5. [Resumen](#5.-Resumen)


## 1. Configuración del entorno
En este taller usaremos también *Pandas*, que es el módulo para tratamiento de datos tabular que más se usa en Python.

Importamos ambos paquetes

In [ ]:
import pandas as pd
import sqlite3

Lo primero que hacemos es **establecer conexión con la base de datos**. En este taller simplemente leeremos la base de datos de un archivo, pero lo normal es que tengamos que configurar la conexión a una base de datos de la empresa. Para ello existen otras librerías como `pyodbc` en el caso de un SQLServer o `cx_Oracle`, si el servidor es Oracle.

**¡Asegurate que tienes el archivo chinook.db en la ruta donde se encuentra este Notebook!**

In [ ]:
import os
os.getcwd()

In [ ]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/chinook.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [ ]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## 2. Modelo de datos
Antes de empezar a atacar una base de datos, tendremos que saber qué hay dentro, y para ello lo mejor es ver cómo es su **modelo de datos**

![imagen](./img/chinook_data_model.png)

Puedes ver todas las tablas que hay en la base de datos con la siguiente sentencia

In [ ]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

## 3. Queries
En las *queries* o consultas, escribiremos nuestra sentencia SQL para poder traernos datos de la base de datos. Las queries tienen esta pinta:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condiciones
```

Por supuesto, hay más sentencias propias de SQL que iremos viendo a lo largo del Notebook. Fíjate que las **palabras reservadas en SQL se suelen poner en mayúsculas**, para diferenciarlas del resto. No da error si se pone de otra manera, ya que SQL **no es *case sensitive***, pero sí se suele hacer así.

### 3.1 Primera query

In [ ]:
query = '''
SELECT * 
FROM tracks
'''

df = sql_query(query)
df

### 3.2 SELECT
**La sentencia `SELECT` se usa para filtrar columnas**. Podremos incluso renombrar algunas columnas dentro del propio `SELECT`. Dos opciones:
* **SELECT * :** de esta forma lo que haremos es traer todos los campos de la tabla. NO es lo recomendable ya que hay tablas que pueden tener cientos de columnas y probablemente no necesitemos todas, por lo que afectará al rendimiento de la maquina y en definitiva al resto de usuarios. **Recuerda que las bases de datos son normalmente entornos compartidos por varios usuarios :)**
* **SELECT campo1, campo2...:** sentencia para escoger los campos concretos que necesitamos.

Además en la propia query podrás renombrar campos mediante la sintaxis `campo as nuevo_nombre`. **Si quieres poner espacios en el nombre del campo, tendrás que rodear el string con comillas dobles**

SQL no es sensible a mayusculas y minusculas.

In [ ]:
query = '''
SELECT Name as "Nombre Cancion", composer as "Compositor"
FROM tracks
'''

sql_query(query)

### 3.3 LIMIT
Se usa para acotar el número de registros de la query. Va siempre al final. Por ejemplo `LIMIT 10`

In [ ]:
query = '''
SELECT Name as "Nombre Cancion", composer
FROM tracks
LIMIT 10
'''

sql_query(query)

### 3.4 DISTINCT
Se usa para obtener todos los registros únicos, es decir, sin duplicados. Muy útil tanto para eliminar dupicados, como para ver todas las casuísticas de un campo en concreto.

**Mucho cuidado con esta sentencia ya que si la tabla tiene miles o millones de registros, puede ralentizar mucho la query.**

In [ ]:
query = '''
SELECT DISTINCT Composer
FROM tracks
'''

sql_query(query)

### 3.5 WHERE
Se usa para filtrar filas. Ejemplos de uso:
* **Un valor numérico**
    * UnitPrice = 0.99
    * UnitPrice >= 0.99
    * UnitPrice < 0.99
* **Un valor string**: Name = 'Restless and Wild'
* **Varios valores**: GenreId in (1, 5, 12)
* **string contenido**:
    * strings que empiecen por 'A': Name like 'A%'
    * strings que acaben en 'A': Name like '%A'
    * strings que lleven 'A' en algun punto: Name like '%A%'
* **Distinto de**: UnitPrice <> 0.99

Probamos el `WHERE`

In [ ]:
query = '''
SELECT * 
FROM tracks
WHERE unitprice > 0.99
'''

sql_query(query)

Veamos qué tal funciona el `LIKE`

In [ ]:
query = '''
SELECT * 
FROM tracks
WHERE composer LIKE '%Brian Johnson%'
'''

sql_query(query)

Compliquemos un poco más el `WHERE`

In [ ]:
query = '''
SELECT * 
FROM tracks
WHERE (unitprice > 0.99 or bytes > 100000000) and genreid in (21, 22, 23) and Name LIKE '%al%'
'''

df = sql_query(query)
df

### 3.6 ORDER BY
Podemos **ordenar la tabla por el campo/s que queramos**. Por defecto ordena alfabéticamente los strings y de menor a mayor los tipos numéricos. Si quieres que ordene al revés, tienes que poner `DESC` de la forma `ORDER BY campo DESC`

In [ ]:
query = '''
SELECT * 
FROM tracks
ORDER BY name DESC;
'''

sql_query(query)

### 3.7 Agregaciones
En ocasiones nos interesa obtener algún estadístico como el máximo de un campo, su desviación estándar o simplemente un conteo de registros no nulos. Para ello podemos usar funciones como `MAX`, `COUNT` o `AVG`. En [esta página](https://www.sqlservertutorial.net/sql-server-aggregate-functions/) encontrarás un resumen con las principales funciones.

In [ ]:
# Aggregations
query = '''
SELECT COUNT(*)
FROM tracks
WHERE name LIKE 'a%';
'''

sql_query(query)

In [ ]:
query = '''
SELECT AVG(unitprice)
FROM invoice_items;
'''

sql_query(query)

### 3.8 GROUP BY
Es una sentencia muy útil para **calcular agregados en función de otro campo**. Por ejemplo, para calcular el total del precio unitario en función del género


In [ ]:
query = '''
SELECT GenreId, SUM(unitprice) as TOT_PRICE
FROM tracks
GROUP BY GenreId
ORDER BY TOT_PRICE DESC
LIMIT 10;
'''

sql_query(query)

O calcular cuantas canciones hay por compositor

In [ ]:
query = '''
SELECT composer, COUNT(trackid)
FROM tracks
WHERE Composer IS NOT NULL
GROUP BY composer
ORDER BY 2 DESC
LIMIT 100
;
'''

sql_query(query)

### 3.9 JOIN
Hasta ahora hemos hecho queries sobre una única tabla, pero **¿y si queremos juntar datos de varias tablas?** Para eso están los `JOIN`s. Para ello **necesitas tener uno o varios campos comunes entre ambas tablas, que denominan CLAVES**.

**¿Cuándo usarlos?** Por ejemplo, si tenemos una tabla con un conjunto de clientes y necesitamos añadirles campos nuevos, tendremos que acudir a otras tablas donde esté ese identificador de cliente y aplicar un `JOIN`. Es lo que se conoce como *pegar campos* a otra tabla.

O imagina que tienes una tabla con todos tus pedidos, con muchos campos(ciudad, dirección, cliente...) y en otra tabla únicamente los números de pedido que no se llegaron a entregar. Si quieres filtrar dentro de tu tabla total de pedidos los que no se llegaron a entregar, podrías aplicar un `INNER JOIN` de manera que te quedes con lo común en ambas tablas, siendo tu clave el identificativo del pedido..

Existen varios tipos de JOINs:

![imagen](./img/joins.jpg)

![imagen](./img/chinook_data_model.png)

In [ ]:
#INNER JOIN
query = '''
SELECT a.name as track_name, a.composer, b.name as genre_name
FROM tracks AS a
INNER JOIN genres AS b
ON a.genreid = b.genreid
;
'''

sql_query(query)

In [ ]:
query = '''
SELECT b.name as genre_name, SUM(a.unitprice) as TOT_PRICE
FROM tracks as a
INNER JOIN genres as b
on a.GenreId=b.GenreId
GROUP BY b.name
ORDER BY TOT_PRICE DESC
LIMIT 10;
'''

sql_query(query)

![imagen](./img/chinook_data_model.png)

In [ ]:

query = '''
SELECT a.trackid, a.name, a.composer, b.invoicelineid, b.invoiceid
FROM tracks AS a
INNER JOIN invoice_items AS b
ON a.trackid = b.trackid
;
'''

sql_query(query)

In [ ]:

query = '''
SELECT name as track_title, albumid
FROM tracks
;
'''

sql_query(query)

In [ ]:
#LEFT JOIN
query = '''
SELECT a.name as track_title , b.title as album_title
FROM tracks AS a
LEFT JOIN albums AS b
ON a.albumid = b.albumid
'''

df_lj = sql_query(query)
# df_lj[df_lj['album_title'].isna()]
df

In [ ]:
query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b
ON a.trackid = b.trackid
LIMIT 5;
'''

sql_query(query)

In [ ]:
query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b
ON a.trackid = b.trackid
UNION
SELECT *
FROM tracks AS a
LEFT JOIN invoice_items AS b
ON a.trackid =b.trackid
LIMIT 5;
'''

sql_query(query)

In [ ]:
#FULL JOIN
#FULL JOIN isn't supported in sqlite, 
#so we use a LEFT JOIN + RIGHT JOIN(inverse LEFT JOIN) as a workaround

query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b
ON a.trackid = b.trackid
UNION
SELECT *
FROM tracks AS a
LEFT JOIN invoice_items AS b
ON a.trackid =b.trackid;
'''

sql_query(query)

### 3.10 VIEW y DROP
En ocasiones resulta útil **dividir nuestras queries en varios pasos**, y para ello utilizamos la sentencia `VIEW`. Con `VIEW` creamos una *vista* de una tabla, es decir, obtenemos una tabla temporal que usaremos después.

Estas vistas se guardarán en el archivo *.db*, por lo que tendrás que tener cuidado de no almacenar muchas tablas poco útiles.

Primero, veamos cómo acceder a todas las vistas de la base de datos

In [ ]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='view'")
for name in res:
    print(name[0])

Creamos dos vistas nuevas

In [ ]:
query1 = '''
CREATE VIEW name_composer as
SELECT name, composer, genreid
FROM tracks
;
'''

query2 = '''
CREATE VIEW genre as
SELECT name, genreid
FROM genres
WHERE name LIKE "b%";
'''

crsr.execute(query1)
crsr.execute(query2)

#sql_query(query)

Comprueba que ahora tienes vistas nuevas en tu base de datos

In [ ]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='view';")
for name in res:
    print(name[0])

Puedes borrar las vistas mediante `DROP VIEW`. Prueba a borrarlas, comprueba que ya no existen y vuelve a crearlas.

**NOTA**: `DROP` también lo podrás usar como `DROP TABLE`. Nos cargaríamos una tabla entera de la base de datos y ya no sería recuperable, el *ctrl + Z* no te va a salvar. así que mucho cuidado con esta sentencia.

In [ ]:
query1_ = '''
DROP VIEW IF EXISTS name_composer;
'''
query2_ = '''
DROP VIEW IF EXISTS genre;
'''

crsr.execute(query1_)
crsr.execute(query2_)

Ahora que ya tenemos nuestras vistas, vamos a realizar otra operación con ambas tablas. En este caso, aplicamos un `JOIN`

Fíjate que la salida de esta query es la misma que la del primer `JOIN` de este Notebook, pero en este caso, hecho en dos pasos.

In [ ]:
query = '''
SELECT a.name, a.composer, b.name
FROM name_composer AS a
INNER JOIN genre AS b
ON a.genreid = b.genreid
;
'''

df = sql_query(query)

In [ ]:
df.to_csv('my_dataset.csv')

## 4. Errores

<table align="left">
 <tr><td width="80"><img src="./img/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES Acceso a base de datos que no existe</h3>
         
 </td></tr>
</table>

¡Cuidado! No da error, sino que crea una base de datos vacía

In [ ]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("bbbb.db")

crsr = connection.cursor()

<table align="left">
 <tr><td width="80"><img src="./img/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES en la query</h3>
         
 </td></tr>
</table>

Los errores son todos del mismo tipo: `OperationalError`. Esto nos indica que lo da el propio SQL, no es un error del intérprete de Python como tal. Ahora bien, aunque lo de SQL, devuelve un descriptivo con el tipo de error, que está bastante bien para ver por donde van los tiros.

In [ ]:
query = '''
SELECT * 
FROM tracks
'''

sql_query(query)

In [ ]:
query = '''
SELECT * 
FROM tracksssss
'''

sql_query(query)

In [ ]:
query = '''
SELECT campo1, campo2
FROM tracks
'''

sql_query(query)

## 5. Resumen
**SQL es el estándar que se utiliza en el acceso a las BBDD relacionales**, por lo que en prácticamente cualquier empresa te vas a encontrar una BD de este tipo. La sintaxis es:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condicionales
```

Y podrás realizar las siguientes acciones:
1. **Filtrar columnas**: mediante `SELECT`
2. **Renombrar campos**: mediante `SELECT campo as nuevo_nombre`
3. **Eliminar duplicados**: mediante `DISTINCT`
4. **Limitar el número de registros**: `LIMIT`
5. **Filtrar filas**: `WHERE`
6. **Ordenar una tabla**: `ORDER BY campo1, campo2` y `DESC`
7. **Agregar información**: sacar un KPI como un máximo o un conteo: `MAX`,`COUNT`,`AVG`...
8. **Agregado a nivel grupo**: con `GROUP BY`. Se calculan las funciones de agregación para cada grupo de una columna.
9. **Juntar datos**: mediante `JOIN`. Hay de varios tipos: `LEFT`, `RIGHT`, `FULL`, `INNER` y `OUTER`.
10. **Tablas temporales**: con `VIEW`
11. **Eliminar tablas o vistas**: con `DROP`

Y si te manejas de maravilla con la librería pandas, **siempre puedes obtener tus datos mediante una query con sqlite3, y tratarlos posteriormente en Python, mediante las funciones de pandas.**